# Проект по SQL.

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.
Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Описание данных.

Загружу нужные библиотеки а потом выведу каждую таблицу с описанием.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [2]:
def sql(query):
    '''Функция передает запрос в формате SQL методу read_sql pandas.
    '''
    return pd.io.sql.read_sql(query, con = engine)

In [3]:
sql('SELECT * FROM books LIMIT 5')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<b>Таблица books.</b>
<p></p>
Содержит данные о книгах:
<li>book_id — идентификатор книги;
<li>author_id — идентификатор автора;
<li>title — название книги;
<li>num_pages — количество страниц;
<li>publication_date — дата публикации книги;
<li>publisher_id — идентификатор издателя.

In [4]:
sql('SELECT * FROM reviews LIMIT 5')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<b>Таблица reviews.</b>
<p></p>
Содержит данные о пользовательских обзорах:
<li>review_id — идентификатор обзора;
<li>book_id — идентификатор книги;
<li>username — имя автора обзора;
<li>text — текст обзора.

In [5]:
sql('SELECT * FROM ratings LIMIT 5')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


<b>Таблица ratings.</b>
<p></p>
Содержит данные о пользовательских оценках книг:
<li>rating_id — идентификатор оценки;
<li>book_id — идентификатор книги;
<li>username — имя пользователя, оставившего оценку;
<li>rating — оценка книги.

In [6]:
sql('SELECT * FROM publishers LIMIT 5')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


<b>Таблица publishers.</b>
<p></p>
Содержит данные об издательствах:
<li>publisher_id — идентификатор издательства;
<li>publisher — название издательства;

In [7]:
sql('SELECT * FROM authors LIMIT 5')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


<b>Таблица authors.</b>
<p></p>
Содержит данные об авторах:
<li>author_id — идентификатор автора;
<li>author — имя автора.

## Задания.
<p></p>
<li>Посчитайте, сколько книг вышло после 1 января 2000 года;
<li>Для каждой книги посчитайте количество обзоров и среднюю оценку;
<li>Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
<li>Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
<li>Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

<b>Посчитайте, сколько книг вышло после 1 января 2000 года.</b>

In [8]:
sql('''
SELECT COUNT(DISTINCT book_id) AS books_count 
FROM books 
WHERE publication_date > '2000-01-01'
''')

,books_count
0,819


<b>Таких книг получилось 819.</b>

<b>Для каждой книги посчитайте количество обзоров и среднюю оценку.</b>

Эти данные можно получить из таблиц reviews и ratings.

In [9]:
sql('''
SELECT reviews.book_id,
       COUNT(DISTINCT(review_id)) AS review_cnt,
       AVG(rating) AS avg_rate
FROM reviews
INNER JOIN ratings ON reviews.book_id=ratings.book_id
GROUP BY reviews.book_id
ORDER BY review_cnt DESC
LIMIT 10
''')

,book_id,review_cnt,avg_rate
0,948,7,3.662500
1,696,6,3.830508
2,207,6,3.395833
3,627,6,3.789474
4,673,6,3.825581
5,695,6,4.081081
6,302,6,4.414634
7,299,6,4.287500
8,497,6,4.125000
9,656,6,4.264151


In [19]:
sql('''
SELECT 
AVG(sub.review_cnt) AS avg_review_count,
AVG(sub.avg_rate) AS avg_rate
FROM
(SELECT reviews.book_id,
       COUNT(DISTINCT(review_id)) AS review_cnt,
       AVG(rating) AS avg_rate
FROM reviews
INNER JOIN ratings ON reviews.book_id=ratings.book_id
GROUP BY reviews.book_id
ORDER BY review_cnt DESC) AS sub
''')

,avg_review_count,avg_rate
0,2.809859,3.898699


<b>В среднем, на одну книгу приходится 2-3 обзора и средняя оценка 3.9. </b>

<b>Определите издательство, которое выпустило наибольшее число книг толще 50 страниц (для исключения из анализа брошюр).</b>

Для этого сформирую запрос в таблице books.

In [21]:
sql('''
SELECT publishers.publisher, 
COUNT(book_id) 
FROM books INNER JOIN publishers ON publishers.publisher_id=books.publisher_id
WHERE num_pages > 50 
GROUP BY publishers.publisher_id 
ORDER BY count DESC LIMIT 5
''')


,publisher,count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


<b>На первом месте издательство Penguin Books с 42 выпущенными книгами толще 50 страниц. На втором месте издательство Vintage и 31 книга, на третьем изательство Grand Central Publishing и 25 книг.</b>

<b>Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.</b>

Для этого объединю таблицы books и ratings  сформулирую запрос.

Таблица с book_id, которые получили больше 50 оценок и самый высокий средний рейтинг. Таких всего 19.

In [13]:
sql('''
SELECT
DISTINCT(book_id),
sub.count_score,
sub.avg_rate
FROM
(SELECT 
book_id,
COUNT(DISTINCT(rating_id)) AS count_score,
AVG(rating) AS avg_rate
FROM ratings
GROUP BY book_id
ORDER BY avg_rate DESC) AS sub
WHERE sub.count_score >= 50

''')

,book_id,count_score,avg_rate
0,75,84,3.678571
1,627,57,3.789474
2,299,80,4.287500
3,79,74,3.729730
4,302,82,4.414634
5,301,75,4.186667
6,545,66,3.787879
7,399,52,4.192308
8,948,160,3.662500
9,722,74,4.391892


По book_id отобранных книг определяю author_id.

In [14]:
sql('''
SELECT
author_id,
sub2.avg_rate,
sub2.count_score
FROM books INNER JOIN 
(SELECT
DISTINCT(book_id),
sub.avg_rate,
sub.count_score
FROM
(SELECT 
book_id,
COUNT(DISTINCT(rating_id)) AS count_score,
AVG(rating) AS avg_rate
FROM ratings
GROUP BY book_id
ORDER BY avg_rate DESC) AS sub
WHERE sub.count_score >= 50) AS sub2
ON books.book_id=sub2.book_id
ORDER BY avg_rate DESC
''')

,author_id,avg_rate,count_score
0,236,4.414634,82
1,240,4.391892,74
2,236,4.287500,80
3,402,4.264151,53
4,236,4.246575,73
5,376,4.192308,52
6,236,4.186667,75
7,240,4.125000,88
8,498,4.080645,62
9,621,3.901408,71


Теперь легко найти имена авторов.

In [15]:
sql('''
SELECT
sub4.author,
AVG(sub4.count_score) AS count_score,
AVG(avg_rate) AS avg_avg_rate
FROM
(SELECT
DISTINCT(author),
sub3.count_score,
sub3.avg_rate
FROM authors INNER JOIN
(SELECT
DISTINCT(author_id),
sub2.avg_rate,
sub2.count_score
FROM books INNER JOIN 
(SELECT
DISTINCT(book_id),
sub.avg_rate,
sub.count_score
FROM
(SELECT 
book_id,
COUNT(DISTINCT(rating_id)) AS count_score,
AVG(rating) AS avg_rate
FROM ratings
GROUP BY book_id
ORDER BY avg_rate DESC) AS sub
WHERE sub.count_score >= 50) AS sub2
ON books.book_id=sub2.book_id) AS sub3
ON authors.author_id=sub3.author_id
ORDER BY avg_rate DESC) AS sub4
GROUP BY sub4.author
ORDER BY avg_avg_rate DESC
''')

,author,count_score,avg_avg_rate
0,J.K. Rowling/Mary GrandPré,77.5,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,53.0,4.264151
2,J.R.R. Tolkien,81.0,4.258446
3,Louisa May Alcott,52.0,4.192308
4,Rick Riordan,62.0,4.080645
5,William Golding,71.0,3.901408
6,J.D. Salinger,86.0,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,57.0,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,66.0,3.787879
9,Dan Brown,71.5,3.754540


<b>Я вывел список всех авторов с количеством оценок больше 50. Так как их немного. Но первое место у J.K. Rowling/Mary GrandPré.</b>

<b>Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.</b>

Найду username пользователей, поставивших больше 50 оценок.

In [16]:
sql('''
SELECT
*
FROM
(SELECT
username,
COUNT(rating_id)
FROM ratings
GROUP BY username) AS sub
WHERE sub.count > 50
''')

,username,count
0,sfitzgerald,55
1,jennifermiller,53
2,xdavis,51
3,paul88,56
4,martinadam,56
5,richard89,55


<b>Таких оказалось всего 6. </b>Теперь посчитаю количество обзоров этих пользователей.

In [17]:
sql('''
SELECT
sub2.username,
COUNT(review_id)
FROM reviews INNER JOIN 
(SELECT
*
FROM
(SELECT
username,
COUNT(rating_id)
FROM ratings
GROUP BY username) AS sub
WHERE sub.count > 50) AS sub2
ON reviews.username=sub2.username
GROUP BY sub2.username
ORDER BY count DESC
''')

,username,count
0,sfitzgerald,28
1,martinadam,27
2,richard89,26
3,jennifermiller,25
4,paul88,22
5,xdavis,18


<b>Самый продуктивный обозреватель - sfitzgerald. У него 28 обзора.</b> Теперь найду среднее.

In [22]:
sql('''
SELECT
AVG(sub3.count) AS avg_reviews
FROM
(SELECT
sub2.username,
COUNT(review_id)
FROM reviews INNER JOIN 
(SELECT
*
FROM
(SELECT
username,
COUNT(rating_id)
FROM ratings
GROUP BY username) AS sub
WHERE sub.count > 50) AS sub2
ON reviews.username=sub2.username
GROUP BY sub2.username
ORDER BY count DESC) AS sub3
''')

,avg_reviews
0,24.333333


<b>Среднее количество обзоров для обозревателей, поставивших больше 50 оценок - 24.</b>

## Выводы:

<p></p>
<li>Количество книг, вышедших после 1 января 2000 года - 819;
<li>Количество обзоров для каждой книги 2-3б средняя оценка книг - 3.9;
<li>Издательство, которое выпустило наибольшее число книг толще 50 страниц — Penguin Books;
<li>Автор с самой высокой средней оценкой книг при условии 50 и более оценок - J.K. Rowling/Mary GrandPré;
<li>Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24.